In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import scale

In [2]:
retail = pd.read_csv("Online+Retail.csv",encoding = "ISO-8859-1")


In [3]:
retail.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [4]:
retail.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [5]:
retail.isnull().sum()
len(retail)

541909

In [6]:
row_count=len(retail)
def count_mising_per(retail):
    mis_val = retail.isnull().sum()
    mis_val_percent = 100 * retail.isnull().sum()/row_count
    mis_val_table=pd.concat([mis_val,mis_val_percent],axis = 1)
    return mis_val_table
        
        

In [7]:
data_missing=count_mising_per(retail)

In [8]:
data_missing=data_missing.rename(columns={0:'count',1:'percent'})

In [9]:
data_missing.sort_values('percent',ascending=False)

,count,percent
CustomerID,135080,24.926694
Description,1454,0.268311
InvoiceNo,0,0.000000
StockCode,0,0.000000
Quantity,0,0.000000
InvoiceDate,0,0.000000
UnitPrice,0,0.000000
Country,0,0.000000


In [10]:
## coustomer id is not relevent so we can drop and description is to low so we can drop one with null

In [11]:
retail.drop(['CustomerID'],axis=1,inplace=True)

In [12]:
retail.dropna(axis=0,inplace=True)

In [13]:
data_missing=count_mising_per(retail)

In [14]:
retail.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,United Kingdom


In [15]:
retail.nunique()

InvoiceNo      24446
StockCode       3958
Description     4223
Quantity         671
InvoiceDate    22309
UnitPrice       1630
Country           38
dtype: int64

In [16]:
len(retail)


540455

In [17]:
retail.drop(['Description','Country'],inplace=True,axis=1)

In [18]:
retail['InvoiceDate']=pd.to_datetime(retail['InvoiceDate'])

In [19]:
retail.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice
0,536365,85123A,6,2010-01-12 08:26:00,2.55
1,536365,71053,6,2010-01-12 08:26:00,3.39
2,536365,84406B,8,2010-01-12 08:26:00,2.75
3,536365,84029G,6,2010-01-12 08:26:00,3.39
4,536365,84029E,6,2010-01-12 08:26:00,3.39


In [20]:
from datetime import date,datetime

In [21]:
def con(a):
    b=datetime.strptime(str(a),"%Y-%m-%d %H:%M:%S")
    return b
retail['inv_date']=retail['InvoiceDate'].apply(con)

In [22]:
retail.head()


,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,inv_date
0,536365,85123A,6,2010-01-12 08:26:00,2.55,2010-01-12 08:26:00
1,536365,71053,6,2010-01-12 08:26:00,3.39,2010-01-12 08:26:00
2,536365,84406B,8,2010-01-12 08:26:00,2.75,2010-01-12 08:26:00
3,536365,84029G,6,2010-01-12 08:26:00,3.39,2010-01-12 08:26:00
4,536365,84029E,6,2010-01-12 08:26:00,3.39,2010-01-12 08:26:00


In [25]:
retail['days']=(datetime.now()-retail['InvoiceDate']).days

AttributeError: 'Series' object has no attribute 'days'

In [58]:
retail['days']=retail['days'].apply(lambda x:x.days)

In [67]:
retail.drop('StockCode',inplace=True,axis=1)

In [69]:
retail.drop('InvoiceNo',inplace=True,axis=1)

In [71]:
retail.drop('InvoiceDate',inplace=True,axis=1)

In [73]:
retail.drop('inv_date',inplace=True,axis=1)

In [74]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
RFM_norm1 = standard_scaler.fit_transform(retail)

C:\Users\vicky\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\vicky\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [75]:
from sklearn.cluster import KMeans

model_clus5 = KMeans(n_clusters = 5, max_iter=50)
model_clus5.fit(RFM_norm1)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=50,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [ ]:
from sklearn.metrics import silhouette_score
sse_ = []
for k in range(2, 15):
    kmeans = KMeans(n_clusters=k).fit(RFM_norm1)
    sse_.append([k, silhouette_score(RFM_norm1, kmeans.labels_)])

In [ ]:
sse_

In [ ]:
ssd = []
for num_clusters in list(range(1,21)):
    model_clus = KMeans(n_clusters = num_clusters, max_iter=50)
    model_clus.fit(RFM_norm1)
    ssd.append(model_clus.inertia_)

plt.plot(ssd)